# Integração entre as Bases RD Station e Exact Sales

> Aqui vamos apontar e resolver os problemas de integração entre as bases [RD Station](https://plugcrm.net/) e [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br), duas bases de CRM utilizadas pela empresa [Absoluta Saúde](https://absolutasaude.com.br/)

## O Problema

O problema a ser resolvido está relacionado à importação da base de _leads_ da base RD para a base Exact, e são eles:

- **Importação incompleta do número de _leads_**: não foram importados todos os leads para a base Exact, sendo que temos 18197 _leads_ na base Exact e 32346 _leads_ na base RD. Assim, _temos 14149 leads que não foram importados para a base Exact_;
- **Importação incompleta das informações dos _leads_**: alguns _leads_ não foram importados com todas as informações para a base Exact, mesmo tendo essas informações na base RD. Alguns casos como: ZANON TECNOLOGIA LTDA ME, VIDAM BRASIL AG DE TURISMO E VIAGENS LTDA entre outras.
    - Uma possível causa desse problema é que pelo formato de saída da exportação do RD do arquivo `.json` ter o _array_ de objetos `custom_fields`, o Exact pode não reconhecer isso da melhor forma na hora da importação

## [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br)

> Temos **18197 leads** nessa base

> - Login: fernando@absolutasaude.com.br
> - Senha: 0i9gky32

### API

- Manual: [Exact Spotter V2 - API v3](https://exactsal.es/apiv3-spotterv2)
- Chave: https://api.exactspotter.com/v3/Leads
- Token: 35fc1bd4-7d6e-4f40-bffd-123d2f46054a

#### Limitações
- Limite de 500 leads por requisição
- Limite de 30 requisições a cada 20 segundos

In [ ]:
# GET Request para leads da Exact

import requests

headers = {
    'Content-Type': 'application/json',
    'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
}

response = requests.get(
    'https://api.exactspotter.com/v3/Leads', headers=headers)

print(response.status_code)
# print(response.headers)
print(response.json())


In [1]:
# Criando base de dados para o Exact

import requests
import json

pagesErrorExct = []
totalLeadsExct = 0

# Funções
# Função para a requisição da API


def request_API(skipNum):
    url = 'https://api.exactspotter.com/v3/Leads'
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }
    params = {
        '$skip': skipNum,
        '$orderby': 'lead'
    }

    result = requests.get(url, headers=headers, params=params)

    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')
        return result

    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        # pagesErrorExct.append(page)

# Função para remover itens da lista


def remove_items(list):
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsExct
        totalLeadsExct = totalLeadsExct + 1

        # Removendo campos desnecessários de cada lead
        # list[i].pop('registerDate')
        list[i].pop('updateDate')
        list[i].pop('mktLink')
        list[i].pop('leadProduct')
        list[i].pop('stage')
        list[i].pop('integrationId')

        # Corrigindo o número de dígitos do CNPJ
        lenCNPJ = len(list[i]["cnpj"])
        if (lenCNPJ < 14):
            if (lenCNPJ == 13):
                list[i]["cnpj"] = "0" + list[i]["cnpj"]
            elif (lenCNPJ == 12):
                list[i]["cnpj"] = "00" + list[i]["cnpj"]
            elif (lenCNPJ == 11):
                list[i]["cnpj"] = "000" + list[i]["cnpj"]
            elif (lenCNPJ == 10):
                list[i]["cnpj"] = "0000" + list[i]["cnpj"]
            elif (lenCNPJ == 9):
                list[i]["cnpj"] = "00000" + list[i]["cnpj"]
            elif (lenCNPJ == 8):
                list[i]["cnpj"] = "000000" + list[i]["cnpj"]

            # Leads sem CNPJ
            # elif (lenCNPJ == 0):
                # print(list[i]['lead'])

            # else:
                # if (lenCNPJ != 0):
                # print(list[i]["cnpj"])
                # print(lenCNPJ)


# Criando a database
listCompleteExct = []

# Usando um número bem alto como 50000 para não correr o risco de não trazer todos os leads
for i in range(0, 30000, 500):
    print('Skip atual:', i)
    # Atualizando a requisição
    responseExct = request_API(i)

    if (responseExct):
        # Transformando em lista (array)
        responseStrExct = json.dumps(responseExct.json()["value"])
        listExct = json.loads(responseStrExct)

        if (listExct):
            print('Skip tem leads')
            # Removendo alguns itens da lista
            remove_items(listExct)

            # Juntando as listas
            listCompleteExct.append(listExct)
            jsonDatabaseExct = json.dumps(listCompleteExct)

        else:
            print('Skip não tem leads')

# Escrevendo no arquivo json
jsonFileExct = open("completeDatabase_Exact.json", "w")
jsonFileExct.write(jsonDatabaseExct)
jsonFileExct.close()

print('Páginas com erro:', pagesErrorExct)
print('Total de leads:', totalLeadsExct)
print("Finalizado")


Skip atual: 0
Requisição: Sucesso
Skip tem leads
Skip atual: 500
Requisição: Sucesso
Skip tem leads
Skip atual: 1000
Requisição: Sucesso
Skip tem leads
Skip atual: 1500
Requisição: Sucesso
Skip tem leads
Skip atual: 2000
Requisição: Sucesso
Skip tem leads
Skip atual: 2500
Requisição: Sucesso
Skip tem leads
Skip atual: 3000
Requisição: Sucesso
Skip tem leads
Skip atual: 3500
Requisição: Sucesso
Skip tem leads
Skip atual: 4000
Requisição: Sucesso
Skip tem leads
Skip atual: 4500
Requisição: Sucesso
Skip tem leads
Skip atual: 5000
Requisição: Sucesso
Skip tem leads
Skip atual: 5500
Requisição: Sucesso
Skip tem leads
Skip atual: 6000
Requisição: Sucesso
Skip tem leads
Skip atual: 6500
Requisição: Sucesso
Skip tem leads
Skip atual: 7000
Requisição: Sucesso
Skip tem leads
Skip atual: 7500
Requisição: Sucesso
Skip tem leads
Skip atual: 8000
Requisição: Sucesso
Skip tem leads
Skip atual: 8500
Requisição: Sucesso
Skip tem leads
Skip atual: 9000
Requisição: Sucesso
Skip tem leads
Skip atual: 9500

## Excluindo Leads Duplicados Exact

> Aqui vamos fazer excluir os leads com o mesmo CNPJ da base do Exact Sales

### Procedimento

- Buscamos os leads com o CNPJ duplicado na base exact e excluímos os com menos informação, deixando somente um


In [2]:
# Excluindo os leads duplicados da base Exact

# Lendo a base Exact
listExctOpen = open("completeDatabase_Exact.json")
listExct = json.load(listExctOpen)

totalLeadsExctDupl = 0
resultExctDupl = []

# Loop correndo para cada CNPJ da base exact, se existe duplicado cria um objeto e insere numa lista
for s in range(0, len(listExct) - 1):
    # for s in range(0, 1):
    print(s)  # 53 no total
    for t in range(0, len(listExct[s])):
        CNPJExct = listExct[s][t]["cnpj"]
        for i in range(0, len(listExct) - 1):
            for j in range(0, len(listExct[i])):
                if ((s, t) != (i, j)):
                    if (listExct[s][t]["cnpj"] and listExct[i][j]["cnpj"]):
                        if (listExct[s][t]["cnpj"] == listExct[i][j]["cnpj"]):
                            resultObj = {
                                'CNPJ': CNPJExct,
                                'Nome Exact': listExct[i][j]["lead"],
                                'Repeticoes': 2
                            }
                            resultExctDupl.append(resultObj)

# Loop para limpar os duplicados mais de uma vez
for i in range(0, len(resultExctDupl) - 1):
    try:
        for j in range(0, len(resultExctDupl)):
            if (i != j):
                if (resultExctDupl[j]["CNPJ"] == resultExctDupl[i]["CNPJ"]):
                    resultExctDupl[j]["Repeticoes"] = resultExctDupl[j]["Repeticoes"] + 1
                    del resultExctDupl[i]
                    break
    except:
        break
for i in range(0, len(resultExctDupl) - 1):
    try:
        for j in range(0, len(resultExctDupl)):
            if (i != j):
                if (resultExctDupl[j]["CNPJ"] == resultExctDupl[i]["CNPJ"]):
                    resultExctDupl[j]["Repeticoes"] = resultExctDupl[j]["Repeticoes"] + 1
                    del resultExctDupl[i]
                    break
    except:
        break

# Loop para contar quantos duplicados exatos existem na base
for i in range(0, len(resultExctDupl)):
    totalLeadsExctDupl = totalLeadsExctDupl + 1

# Loop para excluir os duplicados da base Exact
deletedLeads = []
totalDeletedLeads = 0

for i in range(0, len(resultExctDupl) - 1):
    control = False
    try:
        for j in range(0, len(listExct) - 1):
            for k in range(0, len(listExct[j])):
                if (resultExctDupl[i]["CNPJ"] and listExct[j][k]["cnpj"]):
                    if (resultExctDupl[i]["CNPJ"] == listExct[j][k]["cnpj"]):
                        deletedLeads.append(resultExctDupl[i]["CNPJ"])
                        del listExct[j][k]
                        control = True
                        break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

for i in range(0, len(resultExctDupl) - 1):
    control = False
    try:
        for j in range(0, len(listExct) - 1):
            for k in range(0, len(listExct[j])):
                if (resultExctDupl[i]["CNPJ"] and listExct[j][k]["cnpj"]):
                    if (resultExctDupl[i]["CNPJ"] == listExct[j][k]["cnpj"]):
                        deletedLeads.append(resultExctDupl[i]["CNPJ"])
                        del listExct[j][k]
                        control = True
                        break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

# Loop para contar quantos duplicados exatos foram excluídos da base
for i in range(0, len(listExct)):
    totalDeletedLeads = totalDeletedLeads + 1

# Escrevendo para so arquivos json
resultExctDuplStr = json.dumps(resultExctDupl)
jsonFileExctFinal = open("ExactDuplicateExact.json", "w")
jsonFileExctFinal.write(resultExctDuplStr)
jsonFileExctFinal.close()

deletedLeadsStr = json.dumps(deletedLeads)
jsonFileDeletedLeads = open("exactDeletedLeads.json", "w")
jsonFileDeletedLeads.write(deletedLeadsStr)
jsonFileDeletedLeads.close()

listExctFinal = json.dumps(listExct)
jsonFileListExctFinal = open("exactListFinal.json", "w")
jsonFileListExctFinal.write(listExctFinal)
jsonFileListExctFinal.close()

print("Total de leads duplicados Exact: ", totalLeadsExctDupl)
print("Total de leads excluídos Exact: ", totalDeletedLeads)
print("Finalizado")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
Total de leads duplicados Exact:  3320
Total de leads excluídos Exact:  55
Finalizado


## [RD Station](https://plugcrm.net/)

> Temos **32346 leads** nessa base

> - Login: luciane@absolutasaude.com.br
> - Senha: SgAbs76@380

> - Login: f.aspires@hotmail.com
> - Senha: abs020172

### API

- Manual: [Developers RD Station](https://developers.rdstation.com/reference/instruções-e-requisitos)
- Chave: https://crm.rdstation.com/api/v1/organizations?token=5b67dabde7b4bd01146f3d53
- Token: 5b67dabde7b4bd01146f3d53, Conta f.aspires: 5b67dabde7b4bd01146f3d52

#### Limitações

- Limite de 200 leads por requisição
- Limite de 120 requisições por minuto


In [ ]:
# GET Request para leads da RD

import requests

params = {
    'token': '5b67dabde7b4bd01146f3d53'
}

response = requests.get(
    'https://crm.rdstation.com/api/v1/organizations', params=params)

print(response.status_code)
# print(response.headers)
print(response.json())


In [4]:
# Criando base de dados para o RD

import requests
import json

pagesErrorRd = []
totalLeadsRd = 0

# Funções
# Função para a requisição da API


def request_API(id, page):
    url = 'https://crm.rdstation.com/api/v1/organizations'
    params = {
        'token': '5b67dabde7b4bd01146f3d53',
        'page': page,
        'limit': 500,
        'order': 'name',
        'direction': 'asc',
        'organization_segment': id
    }

    result = requests.get(url, params=params)

    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')
        return result

    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        pagesErrorRd.append(page)

# Função para remover itens da lista


def remove_items(list):
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsRd
        totalLeadsRd = totalLeadsRd + 1

        # Removendo campos desnecessários de cada lead
        list[i].pop('resume')
        list[i].pop('won_count')
        list[i].pop('lost_count')
        list[i].pop('opened_count')
        list[i].pop('paused_count')
        list[i].pop('updated_at')
        list[i].pop('organization_segments')

        if len(list[i]["contacts"]):
            list[i]["contacts"][0].pop('notes')
            list[i]["contacts"][0].pop('facebook')
            list[i]["contacts"][0].pop('linkedin')
            list[i]["contacts"][0].pop('skype')
            list[i]["contacts"][0].pop('birthday')

        if len(list[i]["custom_fields"]):
            # Limpando o campo CNPJ, deixando somente os números
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                    disallowed_characters = "./-"
                    for character in disallowed_characters:
                        if (list[i]["custom_fields"][j]["value"]):
                            list[i]["custom_fields"][j]["value"] = list[i]["custom_fields"][j]["value"].replace(
                                character, "")

                    # Corrigindo o número de dígitos do CNPJ
                    if (list[i]["custom_fields"][j]["value"]):
                        lenCNPJ = len(list[i]["custom_fields"][j]["value"])
                        if (lenCNPJ < 14 and list[i]["custom_fields"][j]["value"]):
                            if (lenCNPJ == 13):
                                list[i]["custom_fields"][j]["value"] = "0" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 12):
                                list[i]["custom_fields"][j]["value"] = "00" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 11):
                                list[i]["custom_fields"][j]["value"] = "000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 10):
                                list[i]["custom_fields"][j]["value"] = "0000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 9):
                                list[i]["custom_fields"][j]["value"] = "00000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 8):
                                list[i]["custom_fields"][j]["value"] = "000000" + \
                                    list[i]["custom_fields"][j]["value"]

            # Excluindo campos desnecessários dos custom fields
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 3"):
                    del [list[i]["custom_fields"][j]]
            # for j in range(0, len(list[i]["custom_fields"]) - 1):
                # if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 2"):
                #     del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 1"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 3"):
                    del [list[i]["custom_fields"][j]]
            # for j in range(0, len(list[i]["custom_fields"]) - 1):
                # if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 1"):
                #     del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL  3  SOCIO 1"):
                    del [list[i]["custom_fields"][j]]
            # for j in range(0, len(list[i]["custom_fields"]) - 1):
                # if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 2"):
                #     del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            # for j in range(0, len(list[i]["custom_fields"]) - 1):
                # if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 2"):
                #     del [list[i]["custom_fields"][j]]
            # for j in range(0, len(list[i]["custom_fields"]) - 1):
                # if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 2"):
                #     del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 3  SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 3"):
                    del [list[i]["custom_fields"][j]]


# Ids dos segmentos
segmentId = [
    '6407ca9e4a0478000c764571',
    '6407ca879a64fa001885e881',
    '6407ca8c53cbb5000c4dcd1b',
    '6407ca92ceb3cb00183993d6',
    '6407ca9988b49100262d7044',
    '640a651c7baa84000c626991',
    '6407caa96ff8ba0028be129f',
    '6407caa4ced60a00123844cc'  
]

# Criando a database
listCompleteRd = []

for i in range(0, len(segmentId)):
    print('ID:', i + 1)
    for j in range(1, 50):  # Usando um número bem alto como 50 para não correr o risco de não trazer todos os leads
        print('Página:', j)
        # Atualizando a requisição
        responseRd = request_API(segmentId[i], j)

        if (responseRd):
            # Transformando em lista (array)
            responseStrRd = json.dumps(responseRd.json()["organizations"])
            listRd = json.loads(responseStrRd)

            if (listRd):
                print('Página tem leads')

                # Removendo alguns itens da lista
                remove_items(listRd)

                # Juntando as listas
                listCompleteRd.append(listRd)
                jsonDatabaseRd = json.dumps(listCompleteRd)

            else:
                print('Página não tem leads')
                break

# Escrevendo no arquivo json
jsonFileRd = open("completeDatabase_RD4.json", "w")
jsonFileRd.write(jsonDatabaseRd)
jsonFileRd.close()

print('Páginas com erro:', pagesErrorRd)
print('Total de leads:', totalLeadsRd)
print("Finalizado")


ID: 1
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: Sucesso
Página tem leads
Página: 3
Requisição: Sucesso
Página tem leads
Página: 4
Requisição: Sucesso
Página tem leads
Página: 5
Requisição: Sucesso
Página tem leads
Página: 6
Requisição: Sucesso
Página tem leads
Página: 7
Requisição: Sucesso
Página tem leads
Página: 8
Requisição: Sucesso
Página tem leads
Página: 9
Requisição: Sucesso
Página tem leads
Página: 10
Requisição: Sucesso
Página tem leads
Página: 11
Requisição: Sucesso
Página tem leads
Página: 12
Requisição: Sucesso
Página tem leads
Página: 13
Requisição: Sucesso
Página tem leads
Página: 14
Requisição: Sucesso
Página não tem leads
ID: 2
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: Sucesso
Página tem leads
Página: 3
Requisição: Sucesso
Página tem leads
Página: 4
Requisição: Sucesso
Página tem leads
Página: 5
Requisição: Sucesso
Página tem leads
Página: 6
Requisição: Sucesso
Página tem leads
Página: 7
Requisição: Sucesso
Página te

## Excluindo Leads Duplicados RD

> Aqui vamos fazer excluir os leads com o mesmo CNPJ da base do RD Station

### Procedimento

- Buscamos os leads com o CNPJ duplicado na base RD e excluímos os com menos informação, deixando somente um


In [4]:
# Excluindo os leads duplicados da base RD

# Lendo a base RD
listRdOpen = open("completeDatabase_RD3.json")
listRd = json.load(listRdOpen)

totalLeadsRdDupl = 0
resultRdDupl = []

# Loop correndo para cada CNPJ da base RD, se existe duplicado cria um objeto e insere numa lista

# for w in range(0, 1):
for w in range(0, len(listRd)):
    print(w)  # 165
    for s in range(0, len(listRd[w])):
        for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
            if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                CNPJRd = listRd[w][s]["custom_fields"][t]["value"]
                for z in range(0, len(listRd)):
                    for i in range(0, len(listRd[z])):
                        for j in range(0, len(listRd[z][i]["custom_fields"]) - 1):
                            if ((s, t) != (i, j)):
                                if (listRd[z][i]["custom_fields"][j]["value"] and listRd[w][s]["custom_fields"][t]["value"]):
                                    if (listRd[z][i]["custom_fields"][j]["value"] == listRd[w][s]["custom_fields"][t]["value"]):
                                        resultObj = {
                                            'CNPJ': CNPJRd,
                                            'Nome RD': listRd[z][i]["name"],
                                            'Repeticoes': 2
                                        }
                                        resultRdDupl.append(resultObj)

# Loop para limpar os duplicados mais de uma vez
for i in range(0, len(resultRdDupl) - 1):
    try:
        for j in range(0, len(resultRdDupl)):
            if (i != j):
                if (resultRdDupl[j]["CNPJ"] == resultRdDupl[i]["CNPJ"]):
                    resultRdDupl[j]["Repeticoes"] = resultRdDupl[j]["Repeticoes"] + 1
                    del resultRdDupl[i]
                    break
    except:
        break
for i in range(0, len(resultRdDupl) - 1):
    try:
        for j in range(0, len(resultRdDupl)):
            if (i != j):
                if (resultRdDupl[j]["CNPJ"] == resultRdDupl[i]["CNPJ"]):
                    resultRdDupl[j]["Repeticoes"] = resultRdDupl[j]["Repeticoes"] + 1
                    del resultRdDupl[i]
                    break
    except:
        break

# Loop para contar quantos duplicados exatos existem na base
for i in range(0, len(resultRdDupl)):
    totalLeadsRdDupl = totalLeadsRdDupl + 1

# Loop para excluir os duplicados da base RD
deletedLeads = []
totalDeletedLeads = 0

for i in range(0, len(resultRdDupl) - 1):
    control = False
    try:
        for w in range(0, len(listRd)):
            for s in range(0, len(listRd[w])):
                for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
                    if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                        if (listRd[w][s]["custom_fields"][t]["value"] == resultRdDupl[i]["CNPJ"]):
                            deletedLeads.append(resultRdDupl[i]["CNPJ"])
                            del listRd[w][s]
                            control = True
                            break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

for i in range(0, len(resultRdDupl) - 1):
    control = False
    try:
        for w in range(0, len(listRd)):
            for s in range(0, len(listRd[w])):
                for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
                    if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                        if (listRd[w][s]["custom_fields"][t]["value"] == resultRdDupl[i]["CNPJ"]):
                            deletedLeads.append(resultRdDupl[i]["CNPJ"])
                            del listRd[w][s]
                            control = True
                            break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

# Loop para contar quantos duplicados exatos foram excluídos da base
for i in range(0, len(listRd)):
    totalDeletedLeads = totalDeletedLeads + 1

# Escrevendo para so arquivos json
resultRdDuplStr = json.dumps(resultRdDupl)
jsonFileRdFinal = open("RdDuplicateLeads.json", "w")
jsonFileRdFinal.write(resultRdDuplStr)
jsonFileRdFinal.close()

deletedLeadsStr = json.dumps(deletedLeads)
jsonFileDeletedLeads = open("RdDeletedLeads.json", "w")
jsonFileDeletedLeads.write(deletedLeadsStr)
jsonFileDeletedLeads.close()

listRdFinal = json.dumps(listRd)
jsonFileListRdFinal = open("RdListFinal.json", "w")
jsonFileListRdFinal.write(listRdFinal)
jsonFileListRdFinal.close()

print("Total de leads duplicados RD: ", totalLeadsRdDupl)
print("Total de leads excluídos RD: ", totalDeletedLeads)
print("Finalizado")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
Total de leads duplicados RD:  1851
Total de leads excluídos RD:  165
Finalizado


## De/Para das Bases

> Aqui vamos fazer o de/para da base do RD Station para a base do Exact Sales e vice-versa

> **Total Leads Exact**: 27152

> **Total Leads RD**: 22822 (com pendência sobre o número total de leads via requisição)

### As bases

- Temos as duas bases de dados prontas, que foram extraídas via requisição na API de cada base
  - A base do RD Station foi tratada para ter no campo _custom_field_ ligado ao _CNPJ_ somente números, já que na extração original os dados vinham com caracteres especiais (,./-)
- Ambas as bases foram tratadas para que tenham sempre os 14 dígitos do CNPJ, tornando possível o abate das informações corretamente

### Procedimento

- Agora, vamos pesquisar para cada lead da base RD se ela existe na base Exact utilizando o método _put_ na requisição
  - **Se existe**: atualizar os dados que não estão preenchidos
  - **Se não existe**: criar o lead


In [6]:
# Fazendo de/para da base exact/rd

# Lendo a base Exact
listExctOpen = open("./databases/exactListFinal.json")
listExct = json.load(listExctOpen)

# Lendo a base Rd
listRdOpen = open("completeDatabase_RD3.json")
listRd = json.load(listRdOpen)

totalLeadsExct = 27155
totalLeadsDePara = 0
totalLeadsNotDePara = 0

resultDePara = []
resultNotDePara = []

# Loop correndo para cada CNPJ da base exact e conferindo se existe na base rd
for s in range(0, len(listExct) - 1):
    print(s)  # 55
    for t in range(0, len(listExct[s])):
        CNPJExct = listExct[s][t]["cnpj"]
        for w in range(0, len(listRd)):
            for i in range(0, len(listRd[w])):
                for j in range(0, len(listRd[w][i]["custom_fields"]) - 1):
                    if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                        res = {
                            k: v for k, v in listRd[w][i]["custom_fields"][j].items() if v == CNPJExct}

                        if (res):
                            resultDeParaObj = {
                                'CNPJ': CNPJExct,
                                'Nome Exact': listExct[s][t]["lead"],
                                'Nome RD': listRd[w][i]["name"],
                                'ID Exact': listExct[s][t]["id"],
                                'ID RD': listRd[w][i]["id"]
                            }
                            resultDePara.append(resultDeParaObj)

                            totalLeadsDePara = totalLeadsDePara + 1

resultDeParaStr = json.dumps(resultDePara)
jsonFileDePara = open("DePara.json", "w")
jsonFileDePara.write(resultDeParaStr)
jsonFileDePara.close()

print("Total de leads De/Para: ", totalLeadsDePara)
print("Finalizado")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
Total de leads De/Para:  13511
Finalizado


# Atualizando Leads

> Atualizando leads no exact que já tem no RD

In [1]:
# Atualizando leads no exact que já tem no RD

import requests
import json

# Lendo arquivos
listOpen = open("./results/DePara.json")
dePara = json.load(listOpen)

listOpen = open("databases/completeDatabase_RD3.json")
listRd = json.load(listOpen)

listOpen = open("databases/exactListFinal.json")
listExct = json.load(listOpen)

# Funções
# Função para a requisição da API


def request_API(values, id):
    url = 'https://api.exactspotter.com/v3/LeadsUpdate/' + str(id)
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }

    result = requests.put(url, data=str(values), headers=headers)

    global status
    global resultJson

    status = result.status_code
    resultJson = result.json()

    print(resultJson)


def replace_unicode(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("À", "A")
        word = word.replace("Á", "A")
        word = word.replace("Â", "A")
        word = word.replace("Ã", "A")
        word = word.replace("Ä", "A")
        word = word.replace("Å", "A")
        word = word.replace("Æ", "A")
        word = word.replace("Ç", "C")
        word = word.replace("È", "E")
        word = word.replace("É", "E")
        word = word.replace("Ê", "E")
        word = word.replace("Ë", "E")
        word = word.replace("Ì", "I")
        word = word.replace("Í", "I")
        word = word.replace("Î", "I")
        word = word.replace("Ï", "I")
        word = word.replace("Ð", "D")
        word = word.replace("Ñ", "N")
        word = word.replace("Ò", "O")
        word = word.replace("Ó", "O")
        word = word.replace("Ô", "O")
        word = word.replace("Õ", "O")
        word = word.replace("Ö", "O")
        word = word.replace("×", "X")
        word = word.replace("Ø", "O")
        word = word.replace("Ù", "U")
        word = word.replace("Ú", "U")
        word = word.replace("Û", "U")
        word = word.replace("Ü", "U")
        word = word.replace("Ý", "Y")
        word = word.replace("Þ", "P")
        word = word.replace("ß", "B")
        word = word.replace("à", "a")
        word = word.replace("á", "a")
        word = word.replace("â", "a")
        word = word.replace("ã", "a")
        word = word.replace("ä", "a")
        word = word.replace("å", "a")
        word = word.replace("æ", "a")
        word = word.replace("ç", "c")
        word = word.replace("è", "e")
        word = word.replace("é", "e")
        word = word.replace("ê", "e")
        word = word.replace("ë", "e")
        word = word.replace("ì", "i")
        word = word.replace("í", "i")
        word = word.replace("î", "i")
        word = word.replace("ï", "i")
        word = word.replace("ð", "o")
        word = word.replace("ñ", "n")
        word = word.replace("ò", "o")
        word = word.replace("ó", "o")
        word = word.replace("ô", "o")
        word = word.replace("õ", "o")
        word = word.replace("ö", "o")
        word = word.replace("÷", "o")
        word = word.replace("ø", "o")
        word = word.replace("ù", "u")
        word = word.replace("ú", "u")
        word = word.replace("û", "u")
        word = word.replace("ü", "u")
        word = word.replace("ý", "y")
        word = word.replace("þ", "p")
        word = word.replace("ÿ", "y")

        return word

def replace_address(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("SAO", "SÃO")
        word = word.replace("JOSE", "JOSÉ")

        return word

def replace_phone(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("+55", "")
        word = word.replace("+", "")
        word = word.replace(" ", "")
        word = word.replace("-", "")

        return word

# Loop correndo todos os leads do de/para buscando informação na base do rd e fazendo a requisição para inserir no exact
totalSuccess = 0
totalFailure = 0
totalInput = 0
finalSuccess = []
finalFailure = []
status = 0
control = 0

for k in range(0, len(dePara)):
    print(k)  # 13511
    for s in range(0, len(listExct)):
        for t in range(0, len(listExct[s])):
            # Pegando valores do exact
            CNPJExct = listExct[s][t]["cnpj"]
            IDExct = listExct[s][t]["id"]
            cel1Exact = listExct[s][t]["phone1"]
            cel2Exact = listExct[s][t]["phone2"]
            addressExact = listExct[s][t]["street"]
            cityExact = listExct[s][t]["city"]
            stateExact = listExct[s][t]["state"]

            for w in range(0, len(listRd)):
                for i in range(0, len(listRd[w])):
                    for j in range(0, len(listRd[w][i]["custom_fields"])):
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                            res = {
                                k: v for k, v in listRd[w][i]["custom_fields"][j].items() if v == CNPJExct}
                        if (res):
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 1"):
                                resCel1 = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 1"):
                                resCel2 = listRd[w][i]["custom_fields"][j]["value"]
                                control = 1
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "Endereço Completo"):
                                resAddress = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CIDADE"):
                                resCity = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "UF"):
                                resState = listRd[w][i]["custom_fields"][j]["value"]
                        
                        if (CNPJExct == "01490046000107"):
                            input("Último CNPJ do De/Para")

                        # Se acha um CNPJ no RD, procura os dados pro objeto de requisição
                        if (res):                            
                            # Construindo objeto e fazendo requisição
                            try:
                                # if (res and resCel1 and addressExact == "" and str(cel1Exact) == "None" and str(cel2Exact) == "None"):
                                if (res and control == 1):                                    
                                    phone = (str(resCel1))
                                    # if (phone == 'None'):
                                    #     phone = (str(cel1Exact))

                                    phone2 = (str(resCel2))
                                    # if (phone2 == 'None'):
                                    #     phone2 = (str(cel2Exact))

                                    address = (str(resAddress))
                                    # if (address == 'None'):
                                    #     address = (str(addressExact))

                                    city = (str(resCity))
                                    # if (city == 'None'):
                                    #     city = (str(cityExact))

                                    state = (str(resState))
                                    # if (state == 'None'):
                                    #     state = (str(stateExact))

                                    cpfcnpj = (str(res["value"]))

                                    # print(resCel1, resCel2, cel1Exact, cel2Exact, address, cityExact, stateExact)

                                    if (str(cel1Exact) == 'None' and str(cel2Exact) == 'None' and addressExact == ""):
                                        result = {
                                            # "duplicityValidation": "true",
                                            "lead":
                                            {
                                                "phone": replace_phone(phone),
                                                "phone2": replace_phone(phone2),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "state": replace_unicode(state),
                                                "country": "Brasil",
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }
                                    
                                    elif (str(cel1Exact) == 'None' and str(cel2Exact) == 'None'):
                                        result = {
                                            # "duplicityValidation": "true",
                                            "lead":
                                            {
                                                "phone": replace_phone(phone),
                                                "phone2": replace_phone(phone2),
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                    else:
                                        resCel1 = ""
                                        resCel2 = ""
                                        resAddress = ""
                                        resCity = ""
                                        resState = ""
                                        resDistrict = ""

                                        name = ""
                                        email = ""
                                        phone = ""
                                        phone2 = ""
                                        website = ""
                                        address = ""
                                        city = ""
                                        state = ""
                                        cpfcnpj = ""

                                        control = 0

                                        break

                                        # result = {
                                        #     # "duplicityValidation": "true",
                                        #     "lead":
                                        #     {
                                        #         "cpfcnpj": replace_unicode(cpfcnpj)
                                        #     }
                                        # }

                                    print(IDExct, result)
                                    # input("Total de 13512 leads")

                                    # Requisição de PUT
                                    request_API(result, IDExct)

                                    if (status == 201):
                                        totalSuccess += 1
                                        finalSuccess.append(IDExct)

                                        resultSuccStr = json.dumps(finalSuccess)
                                        jsonFileSucc = open("AtualizacaoSuccess.json", "w")
                                        jsonFileSucc.write(resultSuccStr)
                                        jsonFileSucc.close()
                                    else:
                                        totalFailure += 1

                                        resultFail = {                                            
                                            "id Exact": IDExct,
                                            "Erro": resultJson,
                                            "duplicityValidation": "true",
                                            "lead":
                                            {
                                                "phone": replace_unicode(phone),
                                                "phone2": replace_unicode(phone2),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "state": replace_unicode(state),
                                                "country": "Brasil",
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                        finalFailure.append(resultFail)

                                        resultFailStr = json.dumps(finalFailure)
                                        jsonFileFail = open("AtualizacaoFail.json", "w")
                                        jsonFileFail.write(resultFailStr)
                                        jsonFileFail.close()


                                    resCel1 = ""
                                    resCel2 = ""
                                    resAddress = ""
                                    resCity = ""
                                    resState = ""
                                    resDistrict = ""

                                    name = ""
                                    email = ""
                                    phone = ""
                                    phone2 = ""
                                    website = ""
                                    address = ""
                                    city = ""
                                    state = ""
                                    cpfcnpj = ""

                                    control = 0

                                    break

                            except:
                                continue

print('Sucesso Total:', totalSuccess)
print('Falha Total:', totalFailure)

print("Finalizado")


0
6590418 {'lead': {'phone': '', 'phone2': '11999255550', 'address': '', 'city': '', 'state': '', 'country': 'Brasil', 'cpfcnpj': '15199026000188'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 6590418}
8711705 {'lead': {'phone': '', 'phone2': '', 'cpfcnpj': '24718073000181'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 8711705}
6611336 {'lead': {'phone': '11999584451', 'phone2': '11969160981', 'cpfcnpj': '04660162000107'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 6611336}
7252784 {'lead': {'phone': '', 'phone2': '', 'cpfcnpj': '23521518000176'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 7252784}
7469502 {'lead': {'phone': '', 'phone2': '', 'cpfcnpj': '08063023000157'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 7469502}
5782046 {'lead': {'

# Criando Leads Exact

> Criando leads que existem na base RD e não existem na base Exact

- Será feita uma requisição para a criação do lead (`request_API`) e outra requisição para adicionar os contatos com o `id` do lead criado (`request_API_Contacts`)
- Para a página do *Lead/Oportunidade* principal, serão adicionados os números de celular vindos da base do RD:
    - Celular 1
    - Cel 1 Sócio 1
- Para o perfil de *Contato*, serão adicionados os números de celular vindos da base do RD:
    - Contacts -> Phone
    - Cel 3 Sócio 3

In [1]:
# Criando leads na exact

import requests
import json

# Lendo arquivos
listOpen = open("databases/completeDatabase_RD4.json")
listRd = json.load(listOpen)

listOpen = open("databases/exactListFinal.json")
listExct = json.load(listOpen)

# Funções
# Função para a requisição da API


def request_API(values):    
    url = 'https://api.exactspotter.com/v3/LeadsAdd'
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }

    result1 = requests.post(url, data=str(values), headers=headers)


    global status
    global newId
    global resultJson

    status = result1.status_code
    resultJson = result1.json()

    print(resultJson)
    
    if (status == 201):
        newId = result1.json()["value"]

    print('Criando lead', status)
    # print(result.headers)

def request_API_Contacts(values):
    url = 'https://api.exactspotter.com/v3/personsAdd'
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }

    result2 = requests.post(url, data=str(values), headers=headers)

    global statusContact
    statusContact = result2.status_code

    print('Inserindo contato', statusContact)

def replace_unicode(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("''", "")

        word = word.replace("À", "\xc3\x80")
        word = word.replace("Á", "\xc3\x81")
        word = word.replace("Â", "\xc3\x82")
        word = word.replace("Ã", "\xc3\x83")
        word = word.replace("Ä", "\xc3\x84")
        word = word.replace("Å", "\xc3\x85")
        word = word.replace("Æ", "\xc3\x86")
        word = word.replace("Ç", "\xc3\x87")
        word = word.replace("È", "\xc3\x88")
        word = word.replace("É", "\xc3\x89")
        word = word.replace("Ê", "\xc3\x8a")
        word = word.replace("Ë", "\xc3\x8b")
        word = word.replace("Ì", "\xc3\x8c")
        word = word.replace("Í", "\xc3\x8d")
        word = word.replace("Î", "\xc3\x8e")
        word = word.replace("Ï", "\xc3\x8f")
        word = word.replace("Ð", "\xc3\x90")
        word = word.replace("Ñ", "\xc3\x91")
        word = word.replace("Ò", "\xc3\x92")
        word = word.replace("Ó", "\xc3\x93")
        word = word.replace("Ô", "\xc3\x94")
        word = word.replace("Õ", "\xc3\x95")
        word = word.replace("Ö", "\xc3\x96")
        word = word.replace("×", "\xc3\x97")
        word = word.replace("Ø", "\xc3\x98")
        word = word.replace("Ù", "\xc3\x99")
        word = word.replace("Ú", "\xc3\x9a")
        word = word.replace("Û", "\xc3\x9b")
        word = word.replace("Ü", "\xc3\x9c")
        word = word.replace("Ý", "\xc3\x9d")
        word = word.replace("Þ", "\xc3\x9e")
        word = word.replace("ß", "\xc3\x9f")
        word = word.replace("à", "\xc3\xa0")
        word = word.replace("á", "\xc3\xa1")
        word = word.replace("â", "\xc3\xa2")
        word = word.replace("ã", "\xc3\xa3")
        word = word.replace("ä", "\xc3\xa4")
        word = word.replace("å", "\xc3\xa5")
        word = word.replace("æ", "\xc3\xa6")
        word = word.replace("ç", "\xc3\xa7")
        word = word.replace("è", "\xc3\xa8")
        word = word.replace("é", "\xc3\xa9")
        word = word.replace("ê", "\xc3\xaa")
        word = word.replace("ë", "\xc3\xab")
        word = word.replace("ì", "\xc3\xac")
        word = word.replace("í", "\xc3\xad")
        word = word.replace("î", "\xc3\xae")
        word = word.replace("ï", "\xc3\xaf")
        word = word.replace("ð", "\xc3\xb0")
        word = word.replace("ñ", "\xc3\xb1")
        word = word.replace("ò", "\xc3\xb2")
        word = word.replace("ó", "\xc3\xb3")
        word = word.replace("ô", "\xc3\xb4")
        word = word.replace("õ", "\xc3\xb5")
        word = word.replace("ö", "\xc3\xb6")
        word = word.replace("÷", "\xc3\xb7")
        word = word.replace("ø", "\xc3\xb8")
        word = word.replace("ù", "\xc3\xb9")
        word = word.replace("ú", "\xc3\xba")
        word = word.replace("û", "\xc3\xbb")
        word = word.replace("ü", "\xc3\xbc")
        word = word.replace("ý", "\xc3\xbd")
        word = word.replace("þ", "\xc3\xbe")
        word = word.replace("ÿ", "\xc3\xbf")

        return word


# Loop correndo todos os leads do de/para buscando informação na base do rd e fazendo a requisição para inserir no exact
totalSuccess = 0
totalFailure = 0
finalSuccess = []
finalFailure = []
status = 0
newId = 0

# Loop Rd
for w in range(0, len(listRd)):
    print('RD', w)  # 139
    for i in range(0, len(listRd[w])):
        for j in range(0, len(listRd[w][i]["custom_fields"]) - 1):
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                CNPJRd = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 1"):
                resCel1 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 2"):
                resCel2 = listRd[w][i]["custom_fields"][j]["value"]

            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 1"):
                resNameSocio1 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 1"):
                resCel1Socio1 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 1"):
                resCel2Socio1 = listRd[w][i]["custom_fields"][j]["value"]

            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 2"):
                resNameSocio2 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 2"):
                resCel1Socio2 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 2"):
                resCel2Socio2 = listRd[w][i]["custom_fields"][j]["value"]

            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CEL  3  SOCIO 3"):
                resCel3 = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "Endereço Completo"):
                resAddress = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CIDADE"):
                resCity = listRd[w][i]["custom_fields"][j]["value"]
            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "UF"):
                resState = listRd[w][i]["custom_fields"][j]["value"]

        # Loop Exact
        for s in range(0, len(listExct)):
            # print('Exact', s) # 55
            if (CNPJRd == "96430483000133"):
                input("Último CNPJ")

            for t in range(0, len(listExct[s])):
                res = {
                    k: v for k, v in listExct[s][t].items() if v == CNPJRd}

                if (res):
                    break

            # Se acha um CNPJ no RD, procura os dados pro objeto de requisição
            if (res):
                break
            else:
                # Construindo objeto e fazendo requisição
                try:
                    if (len(res) == 0 and (resCel1 or resAddress or resCity or resState)):
                        name = (listRd[w][i]["name"])

                        if (resCel1):
                            phone = (str(resCel1))

                        try:
                            if (resCel2):
                                phone2 = (str(resCel2))  
                        except:
                            phone2 = ""

                        try:
                            if (resCel3):
                                phone3 = (str(resCel3))
                        except:
                            phone3 = ""
                        
                        if (resAddress):
                            address = (str(resAddress))
                        
                        if (resCity):
                            city = (str(resCity))
                            if (city == "SAO PAULO"):
                                city = "São Paulo"
                            if (city == "SAO BERNARDO DO CAMPO"):
                                city = "São Bernardo do Campo" 
                            if (city == "SAO JOSE DOS CAMPOS"):
                                city = "São José dos Campos"
                            if (city == "SANTO ANDRE"):
                                city = "Santo André"
                            if (city == "RIBEIRAO PRETO"):
                                city = "Ribeirão Preto"
                            if (city == "MAUA"):
                                city = "Mauá"
                            if (city == "SAO JOSE DO RIO PRETO"):
                                city = "São José do Rio Preto"     
                            if (city == "JUNDIAI"):
                                city = "Jundiaí"
                            if (city == "CARAPICUIBA"):
                                city = "Carapicuíba"
                            if (city == "SAO VICENTE"):
                                city = "São Vicente"
                            if (city == "GUARUJA"):
                                city = "Guarujá"
                            if (city == "TAUBATE"):
                                city = "Taubaté"
                            if (city == "TABOAO DA SERRA"):
                                city = "Taboão da Serra"
                            if (city == "SUMARE"):
                                city = "Sumaré"
                            if (city == "SAO CARLOS"):
                                city = "São Carlos"
                            if (city == "MARILIA"):
                                city = "Marília"
                            if (city == "HORTOLANDIA"):
                                city = "Hortolândia"
                            if (city == "JACAREI"):
                                city = "Jacareí"
                            if (city == "ARACATUBA"):
                                city = "Araçatuba"
                            if (city == "BRAGANCA PAULISTA"):
                                city = "Bragança Paulista"
                            if (city == "SAO CAETANO DO SUL"):
                                city = "São Caetano do Sul"
                            if (city == "MOGI GUACU"):
                                city = "Mogi Guaçu"
                            if (city == "JAU"):
                                city = "Jaú"
                            if (city == "CUBATAO"):
                                city = "Cubatão"

                            if (city == "MAIRIPORA"):
                                city = "Mairiporã"
                            if (city == "POA"):
                                city = "Poá"  
                            if (city == "RIBEIRAO PIRES"):
                                city = "Ribeirão Pires"
                            if (city == "EMBU GUACU"):
                                city = "Embu Guaçu"   
                            if (city == "CARAPICUIBA"):
                                city = "Carapicuíba"
                        
                        if (resState):
                            state = (str(resState))

                        website = (str(listRd[w][i]["url"]))
                        cpfcnpj = (str(CNPJRd))                        

                        # Contruindo objeto para a requisição
                        result = {
                            "duplicityValidation": "true",
                            "lead":
                            {
                                "name": replace_unicode(name),
                                "phone": replace_unicode(phone),
                                "phone2": replace_unicode(phone2),
                                "website": replace_unicode(website),
                                "address": replace_unicode(address),
                                "city": replace_unicode(city),
                                "state": replace_unicode(state),
                                "country": "Brasil",
                                "cpfcnpj": replace_unicode(cpfcnpj)
                            }
                        }

                        IDRd = (listRd[w][i]["id"])

                        print(IDRd, result)
                        input("Press Enter to continue...")

                        # Requisição de POST criando lead
                        request_API(result)

                        if (status == 201):
                            # print("Sucesso: ", totalSuccess)

                            # Valores para o objeto de contato
                            try:
                                try:
                                    emailCtt = (str(listRd[w][i]["contacts"][0]["emails"][0]["email"]))
                                except:
                                    emailCtt = ""
                                
                                try:
                                    nameCtt = (str(resNameSocio1))
                                except:
                                    nameCtt = ""
                                
                                try:
                                    jobTitleCtt = (str(listRd[w][i]["contacts"][0]["title"]))
                                except:
                                    jobTitleCtt = ""
                                
                                try:
                                    phoneCtt = (str(resCel1Socio1))
                                except:
                                    phoneCtt = ""

                                try:
                                    phone2Ctt = (str(resCel2Socio1))
                                except:
                                    phone2Ctt = ""

                                # Criando objeto para inserir contato
                                resultContact = {
                                    "email": replace_unicode(emailCtt),
                                    "name": replace_unicode(nameCtt),
                                    "leadId": (newId),
                                    "jobTitle": replace_unicode(jobTitleCtt),
                                    "phone1": replace_unicode(phoneCtt),
                                    "phone2": replace_unicode(phone2Ctt),
                                    "mainContact": "true"
                                }

                                # Requisição de POST Inserindo Contatos
                                request_API_Contacts(resultContact)
                                
                                # Se tiver dois contatos, insere mais um contato
                                if (resNameSocio2 and resCel1Socio2):
                                    try:
                                        nameCtt = (str(resNameSocio2))
                                    except:
                                        nameCtt = ""

                                    try:
                                        phoneCtt = (str(resCel1Socio2))
                                    except:
                                        phoneCtt = ""
                                    
                                    try:
                                        phone2Ctt = (str(resCel2Socio2))
                                    except:
                                        phone2Ctt = ""

                                    # Criando objeto para inserir contato
                                    resultContact = {
                                        "name": replace_unicode(nameCtt),
                                        "leadId": (newId),
                                        "phone1": replace_unicode(phoneCtt),
                                        "phone2": replace_unicode(phone2Ctt),
                                    }

                                    # Requisição de POST Inserindo Contatos
                                    request_API_Contacts(resultContact)

                                finalSuccess.append(IDRd)

                                resultSuccStr = json.dumps(finalSuccess)
                                jsonFileSucc = open("CriacaoSuccess.json", "w")
                                jsonFileSucc.write(resultSuccStr)
                                jsonFileSucc.close()
                            
                            except:
                                continue

                        else:
                            resultFail = {
                                "idRd": IDRd,
                                "Erro": resultJson,
                                "duplicityValidation": "true",
                                "lead":
                                {
                                    "name": replace_unicode(name),
                                    "phone": replace_unicode(phone),
                                    "phone2": replace_unicode(phone2),
                                    "website": replace_unicode(website),
                                    "address": replace_unicode(address),
                                    "city": replace_unicode(city),
                                    "state": replace_unicode(state),
                                    "country": "Brasil",
                                    "cpfcnpj": replace_unicode(cpfcnpj)
                                }
                            }

                            finalFailure.append(resultFail)

                            resultFailStr = json.dumps(finalFailure)
                            jsonFileFail = open("CriacaoFail.json", "w")
                            jsonFileFail.write(resultFailStr)
                            jsonFileFail.close()

                            finalFailure.append(IDRd)

                        resCel1 = ""
                        resAddress = ""
                        resCity = ""
                        resState = ""
                        resDistrict = ""

                        name = ""
                        email = ""
                        phone = ""
                        website = ""
                        address = ""
                        city = ""
                        state = ""
                        cpfcnpj = ""

                        break

                except:
                    continue

print('Finalizado')


RD 0
609710e93fca9800013477ba {'duplicityValidation': 'true', 'lead': {'name': 'FLC COMUNICACOES LIMITADA AB/21', 'phone': '11991767556', 'phone2': '11991963130', 'website': '', 'address': 'R TILIA 197 TR', 'city': 'OSASCO', 'state': 'SP', 'country': 'Brasil', 'cpfcnpj': '05065043000179'}}
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 12404182}
Criando lead 201
Inserindo contato 201
CARULINA DE LIMA COSTA 11996003739
Inserindo contato 201
609710eb3fca980001347851 {'duplicityValidation': 'true', 'lead': {'name': '02 STUDIO GRAFICO LTDA AB/21', 'phone': '11982579727', 'phone2': '11991963130', 'website': '', 'address': 'R PEDRO COLACO 55 SL 2', 'city': 'SÃ£o Paulo', 'state': 'SP', 'country': 'Brasil', 'cpfcnpj': '05113639000105'}}
